In [ ]:
from stable_baselines3 import PPO
import numpy as np
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from ot2_env_wrapper import OT2Env  # Custom environment wrapper
from clearml import Task  # Import ClearML's Task
import typing_extensions
from stable_baselines3.common.env_util import DummyVecEnv
from stable_baselines3 import PPO  # Replace with your model type
# Paths to your saved models
model_paths = [
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\ksnfka60\model.zip",
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\06lqnlgo\model.zip",
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\8nwi2fpk\model.zip",
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\8102npc5\model.zip",
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\rg8hfw19\model.zip",
    r"C:\Users\Edopi\Desktop\VirtualMLP\models\z4sevf4e\model.zip",
]



def test_model(model_path, env, num_episodes=10):
    try:
        model = PPO.load(model_path)
    except Exception as e:
        print(f"Failed to load model {model_path}: {e}")
        return None, None
    
    distances, rewards = [], []

    for episode in range(num_episodes):
        obs = env.reset()
        done = False
        episode_reward = 0

        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, info = env.step(action)

            # Access the first element of info if using a VecEnv
            info = info[0] if isinstance(info, list) else info

            episode_reward += reward
            if done:
                distances.append(info.get("distance_to_target", 0))
                rewards.append(episode_reward)

    avg_distance = np.mean(distances)
    std_distance = np.std(distances)
    avg_reward = np.mean(rewards)

    print(f"Model: {model_path}, Avg Distance: {avg_distance:.4f}, Std Dev: {std_distance:.4f}, Avg Reward: {avg_reward:.4f}")
    return avg_distance, std_distance



# Main testing routine
def main():
    # Wrap the environment
    def make_env():
        return Monitor(OT2Env(max_steps=1000))

    env = DummyVecEnv([make_env])  # Use DummyVecEnv for compatibility

    results = {}

    # Test each model
    for model_path in model_paths:
        print(f"Testing model: {model_path}")
        avg_distance, std_distance = test_model(model_path, env)
        results[model_path] = {"avg_distance": avg_distance, "std_distance": std_distance}
        print(f"Model: {model_path}, Avg Distance: {avg_distance:.4f}, Std Dev: {std_distance:.4f}")

    # Identify the best model
    best_model = min(results, key=lambda x: results[x]["avg_distance"])
    print(f"\nBest Model: {best_model}")
    print(f"Performance: Avg Distance = {results[best_model]['avg_distance']:.4f}, "
          f"Std Dev = {results[best_model]['std_distance']:.4f}")

    env.close()

if __name__ == "__main__":
    main()

Testing model: C:\Users\Edopi\Desktop\VirtualMLP\models\ksnfka60\model.zip
Reset: Pipette Position [0.073  0.0895 0.1195], Goal Position [ 0.22685139 -0.08999411  0.24448434]
Step 1 called with action: [ 0.9499971  -0.32192013  1.        ]
Step 2 called with action: [-0.2573438  -0.16128212  1.        ]
Step 3 called with action: [ 1.        -1.         0.3959069]
Step 4 called with action: [-0.49503046  0.83333784  0.85629976]
Step 5 called with action: [-0.21730924  0.5588267  -1.        ]
Step 6 called with action: [ 0.14268741 -0.7835681   0.0551929 ]
Step 7 called with action: [-0.90389055  0.7486921  -0.13968876]
Step 8 called with action: [-0.7934716  -0.8495911  -0.83852863]
Step 9 called with action: [ 1.         0.8917929 -1.       ]
Step 10 called with action: [ 1.         -0.72900593  1.        ]
Step 11 called with action: [ 0.37774745 -1.          0.04748458]
Step 12 called with action: [-0.10901035  0.7207257  -1.        ]
Step 13 called with action: [-1.          0.1175

In [ ]:
import matplotlib.pyplot as plt

# Example visualization code
model_names = list(results.keys())
avg_distances = [results[model]["avg_distance"] for model in model_names]

plt.barh(model_names, avg_distances)
plt.xlabel("Average Distance to Target")
plt.ylabel("Model")
plt.title("Model Performance Comparison")
plt.show()
